In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, average_precision_score

In [2]:
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
#from contractions import CONTRACTION_MAP
import unicodedata
#nlp = spacy.load('en_vecs', parse = True, tag=True,) #entity=True)
#nlp_vec = spacy.load('en_vecs', parse = True, tag=True, #entity=True)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

### Load the dataset

In [3]:
blog_df = pd.read_csv('blogtext.csv')

In [4]:
# Selected first 10K rows as the dataset is huge
sub_blog_df = blog_df.head(10000)

### Preprocess rows of the “text” column

In [5]:
# Function to remove special character, remove stopwords and converting to lowercase
# Lemmitization function inside is not working

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def lemmatize_text(text):
    #text = nlp(text)
    #text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text)
    return text

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text


def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
#         if html_stripping:
#             doc = strip_html_tags(doc)
        # remove accented characters
#         if accented_char_removal:
#             doc = remove_accented_chars(doc)
        # expand contractions    
#         if contraction_expansion:
#             doc = expand_contractions(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
             doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [6]:
sub_blog_df['clean_text'] = normalize_corpus(sub_blog_df['text'])

In [7]:
sub_blog_df.head(5)

,id,gender,age,topic,sign,date,text,clean_text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",info found pages mb pdf files wait untill team...
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,team members drewes van der laag urllink mail ...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,thanks yahoo toolbar capture urls popups means...


### Create a dictionary to get the count of every label i.e. the key will be label name and value will
### be the total count of the label. 

In [8]:
sub_blog_df['gender'].value_counts(),sub_blog_df['age'].value_counts(), sub_blog_df['topic'].value_counts(), sub_blog_df['sign'].value_counts() 

(male      5916
 female    4084
 Name: gender, dtype: int64, 35    2315
 36    1708
 17    1185
 27    1054
 24     655
 15     602
 34     553
 16     440
 25     386
 23     253
 26     234
 14     212
 33     136
 39      79
 38      46
 13      42
 37      33
 41      20
 45      16
 42      14
 46       7
 43       6
 44       3
 40       1
 Name: age, dtype: int64, indUnk                     3287
 Technology                 2654
 Fashion                    1622
 Student                    1137
 Education                   270
 Marketing                   156
 Engineering                 127
 Internet                    118
 Communications-Media         99
 BusinessServices             91
 Sports-Recreation            80
 Non-Profit                   71
 InvestmentBanking            70
 Science                      63
 Arts                         45
 Consulting                   21
 Museums-Libraries            17
 Banking                      16
 Automotive                   14


In [156]:
dict(zip(sub_blog_df['gender'].value_counts().index, sub_blog_df['gender'].value_counts().values))

{'male': 5916, 'female': 4084}

In [157]:
dict(zip(sub_blog_df['age'].value_counts().index, sub_blog_df['age'].value_counts().values))

{35: 2315,
 36: 1708,
 17: 1185,
 27: 1054,
 24: 655,
 15: 602,
 34: 553,
 16: 440,
 25: 386,
 23: 253,
 26: 234,
 14: 212,
 33: 136,
 39: 79,
 38: 46,
 13: 42,
 37: 33,
 41: 20,
 45: 16,
 42: 14,
 46: 7,
 43: 6,
 44: 3,
 40: 1}

In [158]:
dict(zip(sub_blog_df['topic'].value_counts().index, sub_blog_df['topic'].value_counts().values))

{'indUnk': 3287,
 'Technology': 2654,
 'Fashion': 1622,
 'Student': 1137,
 'Education': 270,
 'Marketing': 156,
 'Engineering': 127,
 'Internet': 118,
 'Communications-Media': 99,
 'BusinessServices': 91,
 'Sports-Recreation': 80,
 'Non-Profit': 71,
 'InvestmentBanking': 70,
 'Science': 63,
 'Arts': 45,
 'Consulting': 21,
 'Museums-Libraries': 17,
 'Banking': 16,
 'Automotive': 14,
 'Law': 11,
 'LawEnforcement-Security': 10,
 'Religion': 9,
 'Publishing': 4,
 'Accounting': 4,
 'HumanResources': 2,
 'Telecommunications': 2}

In [159]:
dict(zip(sub_blog_df['sign'].value_counts().index, sub_blog_df['sign'].value_counts().values))

{'Aries': 4198,
 'Sagittarius': 1097,
 'Scorpio': 971,
 'Taurus': 812,
 'Aquarius': 571,
 'Cancer': 504,
 'Libra': 491,
 'Pisces': 454,
 'Leo': 301,
 'Virgo': 236,
 'Capricorn': 215,
 'Gemini': 150}

### As we want to make this into a multi-label classification problem, you are required to merge
### all the label columns together, so that we have all the labels together for a particular sentence

In [9]:
interim_df = sub_blog_df.drop(['id','date','text','clean_text'], axis = 1)

In [10]:
sub_blog_df['labels'] = interim_df.values.tolist()
sub_blog_df.head(2)

,id,gender,age,topic,sign,date,text,clean_text,labels
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,team members drewes van der laag urllink mail ...,"[male, 15, Student, Leo]"


In [12]:
# Moved the cleaned text and labels in new dataframe
new_df = sub_blog_df.iloc[:,-2:]
new_df.head(5)

,clean_text,labels
0,info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
1,team members drewes van der laag urllink mail ...,"[male, 15, Student, Leo]"
2,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,testing testing,"[male, 15, Student, Leo]"
4,thanks yahoo toolbar capture urls popups means...,"[male, 33, InvestmentBanking, Aquarius]"


### Separate features and labels, and split the data into training and testing

In [148]:
X = new_df['clean_text']
Y = new_df['labels']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.3, random_state = 5)

### Vectorize the features

In [149]:
vect = CountVectorizer(ngram_range=(1, 2))
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
X_train_dtm, X_test_dtm

(<7000x482311 sparse matrix of type '<class 'numpy.int64'>'
 	with 1036433 stored elements in Compressed Sparse Row format>,
 <3000x482311 sparse matrix of type '<class 'numpy.int64'>'
 	with 273414 stored elements in Compressed Sparse Row format>)

In [15]:
# Getting memory error while running X_train_dtm.toarray()

In [150]:
Y_train

2926                [male, 35, Technology, Aries]
4634            [female, 34, indUnk, Sagittarius]
416                 [female, 24, indUnk, Scorpio]
6369                   [male, 36, Fashion, Aries]
7952          [female, 38, Internet, Sagittarius]
4066                  [female, 25, indUnk, Libra]
3018                [male, 35, Technology, Aries]
2543                [male, 35, Technology, Aries]
9685              [female, 16, Student, Aquarius]
6291               [female, 26, indUnk, Aquarius]
1021                   [male, 23, indUnk, Taurus]
8580                 [male, 16, Internet, Gemini]
5979                 [female, 27, indUnk, Taurus]
8692               [male, 27, Technology, Pisces]
6253                [female, 24, Technology, Leo]
3216                [male, 35, Technology, Aries]
5922                 [female, 27, indUnk, Taurus]
489             [female, 27, Education, Aquarius]
8606            [male, 17, Technology, Capricorn]
342                 [female, 24, indUnk, Scorpio]


In [17]:
# for i in np.arange(0,7000):
#     Y_train[i] = tuple(Y_train[i])
# Y_train

In [152]:
Y_train = [[str(j) for j in i] for i in Y_train]
Y_train

[['male', '35', 'Technology', 'Aries'],
 ['female', '34', 'indUnk', 'Sagittarius'],
 ['female', '24', 'indUnk', 'Scorpio'],
 ['male', '36', 'Fashion', 'Aries'],
 ['female', '38', 'Internet', 'Sagittarius'],
 ['female', '25', 'indUnk', 'Libra'],
 ['male', '35', 'Technology', 'Aries'],
 ['male', '35', 'Technology', 'Aries'],
 ['female', '16', 'Student', 'Aquarius'],
 ['female', '26', 'indUnk', 'Aquarius'],
 ['male', '23', 'indUnk', 'Taurus'],
 ['male', '16', 'Internet', 'Gemini'],
 ['female', '27', 'indUnk', 'Taurus'],
 ['male', '27', 'Technology', 'Pisces'],
 ['female', '24', 'Technology', 'Leo'],
 ['male', '35', 'Technology', 'Aries'],
 ['female', '27', 'indUnk', 'Taurus'],
 ['female', '27', 'Education', 'Aquarius'],
 ['male', '17', 'Technology', 'Capricorn'],
 ['female', '24', 'indUnk', 'Scorpio'],
 ['female', '17', 'indUnk', 'Scorpio'],
 ['male', '35', 'Technology', 'Aries'],
 ['male', '35', 'Technology', 'Aries'],
 ['male', '36', 'Fashion', 'Aries'],
 ['male', '24', 'Engineering', '

### Transform the labels using MultiLabelBinarizer

In [153]:
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(Y_train)
mlb.classes_

array(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', 'Accounting', 'Aquarius', 'Aries', 'Arts',
       'Automotive', 'Banking', 'BusinessServices', 'Cancer', 'Capricorn',
       'Communications-Media', 'Consulting', 'Education', 'Engineering',
       'Fashion', 'Gemini', 'HumanResources', 'Internet',
       'InvestmentBanking', 'Law', 'LawEnforcement-Security', 'Leo',
       'Libra', 'Marketing', 'Museums-Libraries', 'Non-Profit', 'Pisces',
       'Publishing', 'Religion', 'Sagittarius', 'Science', 'Scorpio',
       'Sports-Recreation', 'Student', 'Taurus', 'Technology',
       'Telecommunications', 'Virgo', 'female', 'indUnk', 'male'],
      dtype=object)

In [24]:
y_train

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [26]:
y_test = mlb.transform(Y_test)

In [29]:
y_test

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

### Use OneVsRestClassifier with Logistic Regression

In [46]:
clf = LogisticRegression(solver = 'lbfgs')
clf = OneVsRestClassifier(clf)
clf.fit(X_train_dtm,y_train)
print(accuracy_score(y_train, clf.predict(X_train_dtm)))

0.9561428571428572


### Fit the classifier, make predictions and get the accuracy

In [61]:
print(accuracy_score(y_test, clf.predict(X_test_dtm)))
# Test accuracy is very less cloase to 30%

0.305


In [60]:
print(classification_report(y_test, clf.predict(X_test_dtm), target_names = mlb.classes_))
# Below is classification Report

                         precision    recall  f1-score   support

                     13       1.00      0.22      0.36         9
                     14       0.88      0.09      0.17        76
                     15       0.70      0.20      0.31       190
                     16       0.79      0.24      0.36       127
                     17       0.75      0.25      0.38       334
                     23       0.50      0.01      0.03        70
                     24       0.71      0.11      0.19       182
                     25       0.81      0.12      0.21       108
                     26       0.00      0.00      0.00        67
                     27       0.77      0.37      0.50       331
                     33       1.00      0.32      0.48        44
                     34       1.00      0.65      0.79       176
                     35       0.72      0.62      0.67       711
                     36       0.92      0.52      0.66       527
                     37 

### Comparison of True Label and Predicted Label on Test data

In [143]:
print(list(X_test)[0])
print('True Label', Y_test[0])
pred = clf.predict(X_test_dtm[0])
print('Predicted Label',mlb.inverse_transform(pred))

think best open things im love new smurfalicious stoner cant help tail looks good signed duf
True Label ['male', '36', 'Fashion', 'Aries']
Predicted Label [('36', 'Aries', 'Fashion', 'male')]


In [144]:
print(list(X_test)[1])
print('True Label', Y_test[1])
pred = clf.predict(X_test_dtm[1])
print('Predicted Label',mlb.inverse_transform(pred))

feel fingers feel toes sings glamrock badboy hasbeen billy mack opening sequence love actually sings well ol geezer gyrating rhythm hes not one gyrate vibrate translate prostrate movie no tis season jolly love actually packs punch movie hilarious tho plot intricately weaving various love lives predictably naff understated british humour never ceases make laugh till belly aches id expect nothing less richard curtis man brought us bridget jones diary mr bean black adder series yes rowan atkinson make couple appearances truth told movie really titled lust actually luff actually charts fluffluff lives several characters presentday christmas season london loveatfirstsight loveandsexatsecondsight comes close true meaning love choosing love day despite mistakes close encounters infidelity shies away preferring gloss painful reality favour sweet pure agony love one perks movie offers fact youd recognise faces movie including emma thompson liam neeson colin firth bridget jones alan rickman bad 

In [145]:
print(list(X_test)[2])
print('True Label', Y_test[2])
pred = clf.predict(X_test_dtm[2])
print('Predicted Label',mlb.inverse_transform(pred))

looking forward germany like chemotherapyyou solve one problem expense loosing hair intense vomiting skin boils etc anyway im still healthy case worried tsiona dani thanks calling last night ps america sleep night know around ok ive finally come around accept well life im not waiting something exciting happen im not expecting things radically change im living think may mean ive given something hell parents grandmother insist driving crazy strange ive decided need give much without expecting realize without expecting thanks since nothing enough blows tiddlibum
True Label ['female', '23', 'indUnk', 'Aries']
Predicted Label [('male',)]


In [146]:
print(list(X_test)[3])
print('True Label', Y_test[3])
pred = clf.predict(X_test_dtm[3])
print('Predicted Label',mlb.inverse_transform(pred))

unfortunately madonna correct showing see earlier comments opinion administration handled foreign policy yes harmony going engage part world screaming eagles sometimes people stop listen hit head big stick say military become effective instrument exporting good parts america see dana priests new book mission waging war keeping peace americas military information engaging reading us military successfully stepped delivered peace security places like kosovo institutions like un failed utterly tell ive spending free time another interview today keep fingers crossed
True Label ['male', '35', 'Technology', 'Aries']
Predicted Label [('35', 'Technology', 'male')]


In [147]:
print(list(X_test)[4])
print('True Label', Y_test[4])
pred = clf.predict(X_test_dtm[4])
print('Predicted Label',mlb.inverse_transform(pred))

weeks bob angry flower actually pretty funny
True Label ['male', '35', 'Technology', 'Aries']
Predicted Label [('Aries', 'male')]


In [ ]:
# So here the value predicted are not accurate.